**Model 1D-CNN Class**

In [1]:
import tensorflow as tf 
from tqdm import tqdm  
import numpy as np
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Load the data
with open (r'/home/fede/PhD/UNIGE/Dataset_UniCa/Cagliari/dataset.npy','rb') as f:
  X_train = np.load(f)
  Y_train = np.load(f)
  X_val = np.load(f)
  Y_val = np.load(f)
  X_test =np.load(f)
  Y_test =np.load(f)
  print("Shape of X_train:", X_train.shape)
  print("Shape of y_train:", Y_train.shape)
  print("Shape of y_val:", Y_val.shape)
  print("Shape of y_test:", Y_test.shape)

  X_train = X_train.reshape(X_train.shape[0], 1, 300, 8)
  X_val = X_val.reshape(X_val.shape[0], 1, 300, 8)
  X_test = X_test.reshape(X_test.shape[0], 1, 300, 8)
  
  


2025-02-13 10:19:36.896154: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-13 10:19:36.999387: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-13 10:19:37.521870: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  1
Shape of X_train: (16716, 300, 8)
Shape of y_train: (16716, 6)
Shape of y_val: (3582, 6)
Shape of y_test: (3582, 6)


2025-02-13 10:19:38.428129: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-13 10:19:38.488221: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-13 10:19:38.490615: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
from keras.layers import *
from qkeras import *
from qkeras.qlayers import QDense, QActivation, quantized_bits, quantized_relu
from qkeras import QConv1D
from keras.models import Model
from tensorflow.keras.regularizers import l1

# Define a 1D Convolutional Neural Network (CNN) class
class CNN_1D():
    # Initialize the class with training, validation, and test datasets
    def __init__(self,
                 X_train,
                 y_train,
                 X_val,
                 y_val,
                 X_test,
                 y_test):
        # Model configuration: filters, kernels, batch size, epochs, learning rate, and classes
        self.filters = [[8], [16],[32]]  # Different filter sizes for Conv1D layers
        self.kernels = [2, 3, 4, 5]  # Different kernel sizes for Conv1D layers
        self.batch_size = 64  # Number of samples per gradient update
        self.epochs = 300  # Number of training epochs
        self.learning_rate = 0.001  # Learning rate for the optimizer
        self.n_classes = 6  # Number of output classes
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)  # Optimizer configuration
        self.loss = 'categorical_crossentropy'  # Loss function for classification
        self.metrics = ['accuracy']  # Evaluation metric to track
        # Callbacks for early stopping to prevent overfitting
        self.callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)]
                         # Option to add learning rate scheduling (commented out)

        # Assign the training, validation, and testing data
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.X_test = X_test
        self.y_test = y_test

        # Automatically start training upon initialization
        #self.model = self.train()
        self.model = self.train_q()

    # Internal method to create the CNN model with the specified filters and kernel sizes
    def __create_model(self, filters, kernel, n_classes, verbose=1):
        model = tf.keras.models.Sequential()  # Sequential model for a linear stack of layers
        for i, f in enumerate(filters):
            if i == 0:
                # First Conv1D layer with input shape specified
                model.add(tf.keras.layers.Conv1D(f, kernel, activation='relu', input_shape=(self.X_train.shape[1:]), padding='same'))
            else:
                # Additional Conv1D layers
                model.add(tf.keras.layers.Conv1D(f, kernel, activation='relu', padding='same'))
            model.add(tf.keras.layers.BatchNormalization())  # Batch normalization to stabilize learning
            model.add(tf.keras.layers.MaxPooling1D(2))  # Max pooling to reduce dimensionality
        model.add(tf.keras.layers.GlobalAveragePooling1D())  # Global average pooling layer
        # Output dense layer with softmax activation for classification
        model.add(tf.keras.layers.Dense(n_classes, activation='softmax'))
        if verbose:
            model.summary()  # Print the model architecture if verbose is enabled
        return model


    def create_qmodel(self):
    
        x = x_in = Input(shape= self.X_train.shape[1:], name = "input_layer")

        x = QConv1D(32, 3, name = "q_conv2d", padding='same',kernel_quantizer= quantized_bits(bits=8, integer=4, alpha=1),
            bias_quantizer= quantized_bits(bits=8, integer=4, alpha=1))(x)
        x = QActivation(quantized_relu(bits=8, integer=4, use_sigmoid=0, negative_slope=0.0), name="act_1")(x)
        #x = BatchNormalization()(x)
        x = MaxPooling1D(pool_size=2,name = "max_pool_1")(x)
        x = GlobalAveragePooling1D()(x)        
        x = QDense((6),name = "q_dense",kernel_quantizer= quantized_bits(bits=8, integer=4, alpha=1),
            bias_quantizer= quantized_bits(bits=8, integer=4, alpha=1)) (x)   # num_classes = 10
        x_out = Activation('softmax', name='output_sigmoid')(x)

        model = Model(inputs=[x_in], outputs=[x_out], name='qkeras')

        return model
    def create_qmodel_2d(self):
    
        x = x_in = Input(shape= self.X_train.shape[1:], name = "input_layer")

        x = QConv2D(32, (1, 3), name = "q_conv2d", padding='same',kernel_quantizer= quantized_bits(bits=8, integer=4, alpha=1),
            bias_quantizer= quantized_bits(bits=8, integer=4, alpha=1))(x)
        x = QActivation(quantized_relu(bits=8, integer=4, use_sigmoid=0, negative_slope=0.0), name="act_1")(x)
        #x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(1, 2),name = "max_pool_1")(x)
        x = GlobalAveragePooling2D()(x)        
        x = QDense((6),name = "q_dense",kernel_quantizer= quantized_bits(bits=8, integer=4, alpha=1),
            bias_quantizer= quantized_bits(bits=8, integer=4, alpha=1)) (x)   # num_classes = 10
        x_out = Activation('softmax', name='output_sigmoid')(x)

        model = Model(inputs=[x_in], outputs=[x_out], name='qkeras')

        return model
    def train_q(self):
        
        qmodel = self.create_qmodel_2d()
        initial_optimizer_config = self.optimizer.get_config()
        optimizer = type(self.optimizer).from_config(initial_optimizer_config)  # Reset optimizer for each iteration
        qmodel.compile(optimizer=optimizer, loss=self.loss, metrics=self.metrics)
        qmodel.fit(self.X_train, self.y_train, epochs=self.epochs, batch_size=self.batch_size, 
                                        validation_data=(self.X_val, self.y_val), callbacks=self.callbacks, verbose=1)
        return qmodel

    # Train the CNN model using different combinations of filters and kernels
    def train(self):
        with tf.device('/GPU:0'):
            best_val_acc = 0  # Variable to track the best validation accuracy
            best_f = None  # To store the best filter configuration
            best_k = None  # To store the best kernel size

            # Save initial optimizer state to reuse across models
            initial_optimizer_config = self.optimizer.get_config()

            # Progress bar to display training progress across filter and kernel combinations
            with tqdm(total=len(self.filters) * len(self.kernels), desc="Training Progress", unit="iteration") as pbar:
                for f in self.filters:  # Iterate through each filter configuration
                    for k in self.kernels:  # Iterate through each kernel size
                        # Create and compile the model
                        model = self.__create_model(f, k, self.n_classes, verbose=0)
                        optimizer = type(self.optimizer).from_config(initial_optimizer_config)  # Reset optimizer for each iteration
                        model.compile(optimizer=optimizer, loss=self.loss, metrics=self.metrics)
                        
                        # Train the model on the training data and validate on validation data
                        history = model.fit(self.X_train, self.y_train, epochs=self.epochs, batch_size=self.batch_size, 
                                            validation_data=(self.X_val, self.y_val), callbacks=self.callbacks, verbose=0)
                        
                        # If the current model's validation accuracy is the best, save it
                        if history.history['val_accuracy'][-1] > best_val_acc:
                            best_val_acc = history.history['val_accuracy'][-1]  # Update best validation accuracy
                            model.save(f"model_{f}_{k}.h5")  # Save the model
                            best_f = f  # Track best filter configuration
                            best_k = k  # Track best kernel size
                        pbar.update(1)  # Update the progress bar
            
            # Load the best model based on validation accuracy
            best_model = tf.keras.models.load_model(f"model_{best_f}_{best_k}.h5")

            # Evaluate the best model on training, validation, and testing datasets
            _, train_accuracy = best_model.evaluate(self.X_train, self.y_train, verbose=0)
            _, test_accuracy = best_model.evaluate(self.X_test, self.y_test, verbose=0)
            _, val_accuracy = best_model.evaluate(self.X_val, self.y_val, verbose=0)

            # Print the best filter/kernel configuration and accuracies
            print(f"best_f: {best_f}, best_k: {best_k}")
            print("Best model")
            best_model.summary()  # Print the best model architecture
            print("Train_accuracy:", train_accuracy, "Val accuracy:", val_accuracy, "Test Accuracy:", test_accuracy)

            return best_model

model = CNN_1D(X_train, Y_train, X_val, Y_val, X_test, Y_test)  # Initialize the CNN model

Epoch 1/300
  1/262 [..............................] - ETA: 1:48 - loss: 1.8645 - accuracy: 0.2500

2025-02-13 10:24:03.570735: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape ingradient_tape/qkeras/act_1/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


262/262 [==============================] - 1s 4ms/step - loss: 1.6518 - accuracy: 0.3245 - val_loss: 1.5080 - val_accuracy: 0.4551
Epoch 2/300
262/262 [==============================] - 1s 4ms/step - loss: 1.4146 - accuracy: 0.5028 - val_loss: 1.3198 - val_accuracy: 0.5595
Epoch 3/300
262/262 [==============================] - 1s 3ms/step - loss: 1.2418 - accuracy: 0.5776 - val_loss: 1.1693 - val_accuracy: 0.6044
Epoch 4/300
262/262 [==============================] - 1s 4ms/step - loss: 1.1246 - accuracy: 0.6168 - val_loss: 1.0646 - val_accuracy: 0.6332
Epoch 5/300
262/262 [==============================] - 1s 4ms/step - loss: 1.0260 - accuracy: 0.6477 - val_loss: 0.9943 - val_accuracy: 0.6572
Epoch 6/300
262/262 [==============================] - 1s 5ms/step - loss: 0.9581 - accuracy: 0.6685 - val_loss: 0.9172 - val_accuracy: 0.6776
Epoch 7/300
262/262 [==============================] - 1s 4ms/step - loss: 0.8946 - accuracy: 0.6904 - val_loss: 0.8742 - val_accuracy: 0.6915
Epoch 8/300

In [32]:
import numpy as np
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects

# Load the input data from the C++ test file
input_data = np.loadtxt("/home/fede/test_input_data.txt", dtype=np.float32)

# Reshape to match QKeras input shape
NUM_FEATURES = 8  # Same as in C++
WINDOW_SIZE = 300

input_data = input_data.reshape(1, 1, WINDOW_SIZE, NUM_FEATURES)  # Shape: (1, 300, 8)
#input_data1 = X_train[102].reshape(1, 1, WINDOW_SIZE, NUM_FEATURES) 


print(X_train[0].shape)
# Run the model with the same input
#output_qkeras = model.model.predict(X_train[0])
output_qkeras = model.model.predict(X_train[2:3])  # Keeps batch dimension (1, 1, 300, 8)

print("correct:", X_train[2])

# Print results
max = 0

    
print("QKeras Output:", output_qkeras)
correct_class = np.argmax(Y_train[0])  
print(correct_class)  


(1, 300, 8)
1/1 [==============================] - 0s 15ms/step
correct: [[[ 0.7900269  -0.2052717   0.37117046 ...  0.04761182 -0.5583706
    0.82216614]
  [ 0.71108395 -0.22226508  0.38408306 ...  0.1047499  -0.6087973
    0.78123355]
  [ 0.5852255  -0.4416051   0.01506689 ...  0.12277777 -0.751336
    0.6398249 ]
  ...
  [-0.9766554  -1.3084828   0.8872643  ...  0.523307   -1.5926337
   -0.55269796]
  [-0.9770108  -1.1471397   0.5630894  ...  0.5061585  -1.604404
   -0.5892323 ]
  [-0.98310006 -0.8659002   1.1277664  ...  0.5107626  -1.4530728
   -0.5460629 ]]]
QKeras Output: [[3.2729098e-08 2.5301863e-02 5.4793265e-03 2.2767908e-05 4.2709418e-02
  9.2648661e-01]]
2


In [36]:
def reshape_and_save_input(X_sample, output_file):
    # X_sample should have shape (300, 8), which is (timestep, features)
    X_sample = np.squeeze(X_sample)
    assert X_sample.shape == (300, 8), f"Expected shape (300, 8), but got {X_sample.shape}"

    # Reshape (300, 8) to (1, 8, 1, 300) for Vitis HLS model
    reshaped_data = np.transpose(X_sample)  # Transpose from (300, 8) to (8, 300)
    reshaped_data = reshaped_data.flatten()  # Flatten to a single 1D array (2400 elements)

    # Save reshaped data to a text file
    with open(output_file, 'w') as f:
        for value in reshaped_data:
            f.write(f"{value}\n")

    print(f"Reshaped data saved to {output_file}")


sample_n = 77

output_qkeras = model.model.predict(X_train[sample_n:sample_n + 1])  # Keeps batch dimension (1, 1, 300, 8)

print("sample n:", sample_n)
print( X_train[sample_n])

    
print("QKeras Output:", output_qkeras)
correct_class = np.argmax(Y_train[sample_n])  
print("correct class:",correct_class)  
X_sample = X_train[sample_n]  # Select the first sample (shape: (300, 8))

# File to save reshaped input
output_file = "/home/fede/reshaped_input_vitis.txt"

# Call function to reshape and save input data
reshape_and_save_input(X_sample, output_file)

1/1 [==============================] - 0s 16ms/step
sample n: 77
[[[ 1.3198973   0.43553346 -0.48278418 ... -1.0238531   0.1940515
   -0.7256301 ]
  [ 1.3007689   0.42590395 -0.02840268 ... -0.8660884   0.38284305
   -0.47496957]
  [ 1.3269824   0.06821638 -0.7948421  ... -0.8538679  -0.03489642
   -0.45548582]
  ...
  [ 1.6323969   0.51933026 -1.4022009  ... -1.8409101  -0.55843925
   -2.2259157 ]
  [ 1.6515054   0.74563575 -1.1497234  ... -1.9835336   0.09736931
   -1.7994627 ]
  [ 1.5731585   0.73989815 -0.628648   ... -1.9172432   0.58965504
   -1.0549612 ]]]
QKeras Output: [[9.5840979e-01 3.9589211e-02 1.4513045e-03 5.4628152e-04 5.1536841e-08
  3.3792917e-06]]
correct class: 0
Reshaped data saved to /home/fede/reshaped_input_vitis.txt


In [13]:
import os
import numpy as np
import tensorflow as tf

# Save weights and biases for each layer in the specified format
weights_dir = "model_weights_q"
os.makedirs(weights_dir, exist_ok=True)

for i, layer in enumerate(model.model.layers):
    weights = layer.get_weights()
    if weights:  # Only save if the layer has weights
        layer_name = layer.__class__.__name__
        if isinstance(layer, tf.keras.layers.Conv1D) :
            with open(os.path.join(weights_dir, f"Conv_{i}_params.h"), "w") as f:
                f.write(f"#ifndef Conv_{i}_PARAMS\n#define Conv_{i}_PARAMS\n\n")
                f.write(f"#define WEIGHT_Conv_{i} ")
                # Transpose the weights to match [depth][height][width][kernel]
                # weights[0] shape is [filter_height, input_channels, output_channels]
                # We need to transpose it to [input_channels, filter_height, 1, output_channels]
                
                weight_array = np.transpose(weights[0], (1, 0, 2))  # [depth][height][1][kernel]
                print(weights)
                f.write("{" + np.array2string(weight_array, separator=', ', formatter={'float_kind': lambda x: f'{x:.6f}'} , threshold=np.inf)[1:-1].replace('[', '{').replace(']', '}').replace('\n', '') + "}\n")
                f.write(f"#define BIAS_Conv_{i} ")
                f.write("{" + np.array2string(weights[1], separator=', ', formatter={'float_kind': lambda x: f'{x:.6f}'} , threshold=np.inf)[1:-1].replace('[', '{').replace(']', '}').replace('\n', '') + "}\n")
                f.write(f"#endif\n")
        elif isinstance(layer, tf.keras.layers.BatchNormalization):
            with open(os.path.join(weights_dir, f"BatchNorm_{i}_params.h"), "w") as f:
                f.write(f"#ifndef BatchNorm_{i}_PARAMS\n#define BatchNorm_{i}_PARAMS\n\n")
                f.write(f"#define GAMMA_BatchNorm_{i} ")
                f.write("{" + np.array2string(weights[0], separator=', ', formatter={'float_kind': lambda x: f'{x:.6f}'} , threshold=np.inf)[1:-1].replace('[', '{').replace(']', '}').replace('\n', '') + "}\n")
                f.write(f"#define BETA_BatchNorm_{i} ")
                f.write("{" + np.array2string(weights[1], separator=', ', formatter={'float_kind': lambda x: f'{x:.6f}'} , threshold=np.inf)[1:-1].replace('[', '{').replace(']', '}').replace('\n', '') + "}\n")
                f.write(f"#define MOVING_MEAN_BatchNorm_{i} ")
                f.write("{" + np.array2string(weights[2], separator=', ', formatter={'float_kind': lambda x: f'{x:.6f}'} , threshold=np.inf)[1:-1].replace('[', '{').replace(']', '}').replace('\n', '') + "}\n")
                f.write(f"#define MOVING_VARIANCE_BatchNorm_{i} ")
                f.write("{" + np.array2string(weights[3], separator=', ', formatter={'float_kind': lambda x: f'{x:.6f}'} , threshold=np.inf)[1:-1].replace('[', '{').replace(']', '}').replace('\n', '') + "}\n")
                f.write(f"#endif\n")
        elif isinstance(layer, tf.keras.layers.Dense):
            with open(os.path.join(weights_dir, f"Dense_{i}_params.h"), "w") as f:
                f.write(f"#ifndef Dense_{i}_PARAMS\n#define Dense_{i}_PARAMS\n\n")
                f.write(f"#define WEIGHT_Dense_{i} ")
                f.write("{" + np.array2string(weights[0], separator=', ', formatter={'float_kind': lambda x: f'{x:.6f}'} , threshold=np.inf)[1:-1].replace('[', '{').replace(']', '}').replace('\n', '') + "}\n")
                f.write(f"#define BIAS_Dense_{i} ")
                f.write("{" + np.array2string(weights[1], separator=', ', formatter={'float_kind': lambda x: f'{x:.6f}'} , threshold=np.inf)[1:-1].replace('[', '{').replace(']', '}').replace('\n', '') + "}\n")
                f.write(f"#endif\n")

print(f"Weights and biases saved to {weights_dir} in specified format")


KeyboardInterrupt: 

In [12]:
from qonnx.converters import from_keras

output_path = '/home/fede/PhD/UNIGE/ONNX'

path = output_path + '/qonnx_model_unige.onnx'
print("conversion to qonnx...")
qonnx_model, _  = from_keras(
    model.model,
    name="qkeras_to_qonnx_converted",
    input_signature=None,
    opset=None,
    custom_ops=None,
    custom_op_handlers=None,
    custom_rewriter=None,
    inputs_as_nchw=None,
    extra_opset=None,
    shape_override=None,
    target=None,
    large_model=False,
    output_path = path,
)

conversion to qonnx...


2025-02-13 10:47:36.295631: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-13 10:47:36.296795: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2025-02-13 10:47:36.296905: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2025-02-13 10:47:36.297198: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-02-13 10:47:36.298302: I tensorflow/compiler/xla/stream_executor/